## import necessary package

In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import kagglehub

# general setting. do not change TEST_SIZE
RANDOM_SEED = 42
TEST_SIZE = 0.3

## load dataset & prepare data
Class欄位(是否為詐欺交易)轉換成整數型態

資料預處理，刪除較無關的time欄位，對Amount(金額)欄位做標準化(因為金額數字變化範圍太大會影響模型學習效果)

In [2]:
# load dataset（from kagglehub）
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")
data = pd.read_csv(f"{path}/creditcard.csv")
data['Class'] = data['Class'].astype(int)

# prepare data
data = data.drop(['Time'], axis=1)
data['Amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))

## Fraud/Non-Fraud Transactions
統計詐欺與非詐欺交易的數量與比例(因為欺騙樣本極少，所以模型會傾向預測非詐欺)

In [3]:
fraud = data[data['Class'] == 1]
nonfraud = data[data['Class'] == 0]
print(f'Fraudulent:{len(fraud)}, non-fraudulent:{len(nonfraud)}')
print(f'the positive class (frauds) percentage: {len(fraud)}/{len(fraud) + len(nonfraud)} ({len(fraud)/(len(fraud) + len(nonfraud))*100:.3f}%)')

Fraudulent:492, non-fraudulent:284315
the positive class (frauds) percentage: 492/284807 (0.173%)


## 監督式學習(random forest)
模型訓練

In [4]:
X = np.asarray(data.iloc[:, ~data.columns.isin(['Class'])])
Y = np.asarray(data.iloc[:, data.columns == 'Class'])

# # split training set and data set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=TEST_SIZE, random_state=RANDOM_SEED)

Y_train = Y_train.ravel()
Y_test = Y_test.ravel()

# # build Random Forest model
rf_model = RandomForestClassifier(
    n_estimators=100,
    random_state=RANDOM_SEED,
    class_weight={0: 1, 1: 5.5},
    max_features=10
   )

rf_model.fit(X_train, Y_train)



RandomForestClassifier(class_weight={0: 1, 1: 5.5}, max_features=10,
                       random_state=42)

## result of 監督式學習(random forest)

*  Precision精確率：預測出的詐欺交易中有多少為真(猜的準不準 -> 如果比較保守判斷P可以比較高)
*   Recall靈敏度：在所有實際詐欺交易中模型抓到多少(量大的猜容易涵蓋到較多真的詐欺交易 -> 較開放的判斷R會比較好)
*   F1-score綜合指標：Precision 和 Recall 的加權平均（偏向兩者平衡）

In [5]:
# define evaluation function
def evaluation(y_true, y_pred, model_name="Model"):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f'\n{model_name} Evaluation:')
    print('===' * 15)
    print('         Accuracy:', accuracy)
    print('  Precision Score:', precision)
    print('     Recall Score:', recall)
    print('         F1 Score:', f1)
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))

# predict and print result
y_pred = rf_model.predict(X_test)
# print(classification_report(y_test, y_pred))
evaluation(Y_test, y_pred, model_name="Random Forest(Supervised)")


Random Forest(Supervised) Evaluation:
         Accuracy: 0.9996488887328394
  Precision Score: 0.9416666666666667
     Recall Score: 0.8308823529411765
         F1 Score: 0.8828125

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85307
           1       0.94      0.83      0.88       136

    accuracy                           1.00     85443
   macro avg       0.97      0.92      0.94     85443
weighted avg       1.00      1.00      1.00     85443



## 非監督式學習(KMeans)

In [6]:
from sklearn.decomposition import PCA

# Extract features and labels
X = np.asarray(data.drop(columns=['Class']))
y = np.asarray(data['Class'])

# Split the dataset into training and testing sets (with stratification)
x_train, x_test, y_train, y_test = train_test_split(
   X, y, test_size=TEST_SIZE, random_state=RANDOM_SEED, stratify=y
)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# PCA 降維
pca = PCA(n_components=15)  # 嘗試 2-20 的不同值
x_train = pca.fit_transform(x_train)
x_test = pca.transform(x_test)

# Select a small sample of normal (non-fraud) data for unsupervised training
n_x_train = x_train[y_train == 0]
n_x_train = n_x_train[:1000]

scores = []
for k in range(2, 5):
   kmeans = KMeans(
       n_clusters=k,
       init='k-means++',
       random_state=RANDOM_SEED,
       max_iter=500,
       n_init=20
       )
   kmeans.fit(n_x_train)
   score = silhouette_score(n_x_train, kmeans.labels_)
   scores.append(score)

optimal_k = np.argmax(scores) + 2
kmeans = KMeans(
    n_clusters=optimal_k,
    init='k-means++',
    random_state=RANDOM_SEED,
    max_iter=500,
    n_init=20
    )
kmeans.fit(n_x_train)
y_pred_test = kmeans.predict(x_test)
def align_labels(y_true, y_pred, n_clusters):
   labels = np.zeros_like(y_pred)
   for i in range(n_clusters):
       mask = (y_pred == i)
       if np.sum(mask) > 0:
           labels[mask] = np.bincount(y_true[mask]).argmax()
       else:
           labels[mask] = 0  # Default to normal class
   return labels

y_pred_aligned = align_labels(y_test, y_pred_test, optimal_k)

## result of 非監督式學習(KMeans)


In [7]:
def evaluation(y_true, y_pred, model_name="Model"):
   accuracy = accuracy_score(y_true, y_pred)
   precision = precision_score(y_true, y_pred, zero_division=0)
   recall = recall_score(y_true, y_pred)
   f1 = f1_score(y_true, y_pred)

   print(f'\n{model_name} Evaluation:')
   print('===' * 15)
   print('         Accuracy:', accuracy)
   print('  Precision Score:', precision)
   print('     Recall Score:', recall)
   print('         F1 Score:', f1)
   print("\nClassification Report:")
   print(classification_report(y_true, y_pred))

evaluation(y_test, y_pred_aligned, model_name="KMeans (Unsupervised)")


KMeans (Unsupervised) Evaluation:
         Accuracy: 0.9987477031471274
  Precision Score: 0.7887323943661971
     Recall Score: 0.3783783783783784
         F1 Score: 0.5114155251141552

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85295
           1       0.79      0.38      0.51       148

    accuracy                           1.00     85443
   macro avg       0.89      0.69      0.76     85443
weighted avg       1.00      1.00      1.00     85443

